# Load the Pretrained Model and the dataset
We use ernie-2.0-base-en as the model and SST-2 as the dataset for example. More models can be found in [PaddleNLP Model Zoo](https://paddlenlp.readthedocs.io/zh/latest/model_zoo/index.html#transformer).

Obviously, PaddleNLP is needed to run this notebook, which is easy to install:
```bash
pip install setuptools_scm 
pip install --upgrade paddlenlp
```

In [1]:
import paddle
import paddlenlp
from paddlenlp.transformers import ErnieForSequenceClassification, ErnieTokenizer

MODEL_NAME = "ernie-2.0-base-en"

model = ErnieForSequenceClassification.from_pretrained(MODEL_NAME, num_classes=2)
tokenizer = ErnieTokenizer.from_pretrained(MODEL_NAME)

from paddlenlp.datasets import load_dataset
train_ds, dev_ds, test_ds = load_dataset(
    "glue", name='sst-2', splits=["train", "dev", "test"]
)

/opt/conda/envs/pp2/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/pp2/lib/python3.7/site-packages/paddlenlp/transformers/image_utils.py:213: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  resample=Image.BILINEAR,
/opt/conda/envs/pp2/lib/python3.7/site-packages/paddlenlp/transformers/image_utils.py:379: DeprecationWarning: NEAREST is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.NEAREST or Dither.NONE instead.
  resample=Image.NEAREST,
/opt/conda/envs/pp2/lib/python3.7/site-packages/paddlenlp/transformers/ernie_vil/feature_extraction.py:65: DeprecationWarning: BICUBIC is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BICUBIC instead.
  resample=Image.BICUBIC,

# Prepare the Model

## Train the model

In [2]:
# training the model and save to save_dir
# only needs to run once.
# total steps ~2100 (1 epoch)

from assets.utils import training_model
training_model(model, tokenizer, train_ds, dev_ds, save_dir='assets/sst-2-ernie-2.0-en')

dataset labels: ['0', '1']
dataset examples:
{'sentence': 'hide new secretions from the parental units ', 'labels': 0}
{'sentence': 'contains no wit , only labored gags ', 'labels': 0}
{'sentence': 'that loves its characters and communicates something rather beautiful about human nature ', 'labels': 1}
{'sentence': 'remains utterly satisfied to remain the same throughout ', 'labels': 0}
{'sentence': 'on the worst revenge-of-the-nerds clichés the filmmakers could dredge up ', 'labels': 0}
Training Starts:
global step 100, epoch: 1, batch: 100, loss: 0.28690, acc: 0.76781
global step 200, epoch: 1, batch: 200, loss: 0.21684, acc: 0.82969
global step 300, epoch: 1, batch: 300, loss: 0.22296, acc: 0.84958
global step 400, epoch: 1, batch: 400, loss: 0.38831, acc: 0.86469
global step 500, epoch: 1, batch: 500, loss: 0.07305, acc: 0.87375
global step 600, epoch: 1, batch: 600, loss: 0.14774, acc: 0.88099
global step 700, epoch: 1, batch: 700, loss: 0.22830, acc: 0.88732
global step 800, epoc

[2022-09-26 11:46:08,224] [    INFO] - tokenizer config file saved in assets/sst-2-ernie-2.0-en/tokenizer_config.json
[2022-09-26 11:46:08,225] [    INFO] - Special tokens file saved in assets/sst-2-ernie-2.0-en/special_tokens_map.json


## Or Load the trained model

In [2]:
# Load the trained model.
state_dict = paddle.load("assets/sst-2-ernie-2.0-en/model_state.pdparams")
model.set_dict(state_dict)

# See the prediction results

In [3]:
import interpretdl as it
import numpy as np
from assets.utils import aggregate_subwords_and_importances
from paddlenlp.data import Stack, Tuple, Pad
from interpretdl.data_processor.visualizer import VisualizationTextRecord, visualize_text
from assets.utils import predict

reviews = [
    "it 's a charming and often affecting journey . ",
    'the movie achieves as great an impact by keeping these thoughts hidden as ... ( quills ) did by showing them . ',
    'this one is definitely one to skip , even for horror movie fanatics . ',
    'in its best moments , resembles a bad high school production of grease , without benefit of song . '
]

data = [ {"text": r} for r in reviews]

label_map = {0: 'negative', 1: 'positive'}
batch_size = 32

results = predict(
    model, data, tokenizer, label_map, batch_size=batch_size)

for idx, text in enumerate(data):
    print('Data: {} \t Lable: {}'.format(text, results[idx]))

Data: {'text': "it 's a charming and often affecting journey . "} 	 Lable: positive
Data: {'text': 'the movie achieves as great an impact by keeping these thoughts hidden as ... ( quills ) did by showing them . '} 	 Lable: positive
Data: {'text': 'this one is definitely one to skip , even for horror movie fanatics . '} 	 Lable: negative
Data: {'text': 'in its best moments , resembles a bad high school production of grease , without benefit of song . '} 	 Lable: negative


# Prepare for Interpretations

In [4]:
import interpretdl as it 
import numpy as np
from assets.utils import aggregate_subwords_and_importances
from interpretdl.data_processor.visualizer import VisualizationTextRecord, visualize_text

true_labels = [1, 1, 0, 0] * 5
recs = []
reviews = [
    "it 's a charming and often affecting journey . ",
    'the movie achieves as great an impact by keeping these thoughts hidden as ... ( quills ) did by showing them . ',
    'this one is definitely one to skip , even for horror movie fanatics . ',
    'in its best moments , resembles a bad high school production of grease , without benefit of song . '
]

def text_to_input(raw_text):
    encoded_inputs = tokenizer(text=raw_text, max_seq_len=128)
    _batched_and_to_tuple = tuple([np.array([v]) for v in encoded_inputs.values()])
    return _batched_and_to_tuple


## IG Interpreter

In [5]:

ig = it.IntGradNLPInterpreter(model, device='gpu:0')
recs = []
for i, review in enumerate(reviews):
    ig_weights = ig.interpret(
        review,
        # tokenizer=tokenizer,
        text_to_input_fn=text_to_input,
        steps=50
    )
    pred_class = ig.predcited_label[0]
    pred_prob = ig.predcited_proba[0, pred_class]
    
    ig_weights = np.sum(ig_weights, axis=-1)

    # subwords with [CLS] and [SEP]
    # encoded_inputs = tokenizer(review)
    # subwords = tokenizer.convert_ids_to_tokens(encoded_inputs['input_ids'])
    # subwords without special tokens.

    subwords = " ".join(tokenizer._tokenize(review)).split(' ')
    subword_importances = ig_weights[0]
    
    words, word_importances = aggregate_subwords_and_importances(subwords, subword_importances)
    word_importances = np.array(word_importances) / np.linalg.norm(
        word_importances)
    
    true_label = true_labels[i]
    interp_class = pred_class
    
    if interp_class == 0:
        word_importances = -word_importances
        
    rec = VisualizationTextRecord(
        words, 
        word_importances, 
        true_label,                   
        pred_class, 
        pred_prob,
        interp_class
    )
    
    recs.append(rec)

visualize_text(recs)
# The visualization is not available at github

True Label,Predicted Label (Prob),Target Label,Word Importance
1,1 (1.00),1,it ' s a charming and often affecting journey .
1,1 (0.96),1,the movie achieves as great an impact by keeping these thoughts hidden as . . . ( quills ) did by showing them .
0,0 (0.96),0,"this one is definitely one to skip , even for horror movie fanatics ."
0,0 (0.99),0,"in its best moments , resembles a bad high school production of grease , without benefit of song ."


## LIME Interpreter

In [6]:

lime = it.LIMENLPInterpreter(model, device='gpu:0')
recs = []
for i, review in enumerate(reviews):
    lime_weights = lime.interpret(
        review,
        # tokenizer=tokenizer,
        text_to_input=text_to_input,
        num_samples=1000,
        batch_size=50,
        visual=False
    )
    pred_class = list(lime.lime_results['probability'].keys())
    pred_prob = list(lime.lime_results['probability'].values())

    # subwords with [CLS] and [SEP]
    # encoded_inputs = tokenizer(review)
    # subwords = tokenizer.convert_ids_to_tokens(encoded_inputs['input_ids'])
    # subwords without special tokens.
    subwords = " ".join(tokenizer._tokenize(review)).split(' ')
    interp_class = list(lime_weights.keys())[0]
    weights = lime_weights[interp_class][1:-1]
    subword_importances = [t[1] for t in lime_weights[interp_class][1:-1]]
    
    words, word_importances = aggregate_subwords_and_importances(subwords, subword_importances)
    word_importances = np.array(word_importances) / np.linalg.norm(
        word_importances)
    
    true_label = true_labels[i]
    
    if interp_class == 0:
        word_importances = -word_importances
        
    rec = VisualizationTextRecord(
        words, 
        word_importances, 
        true_label,                   
        pred_class[0], 
        pred_prob[0],
        interp_class
    )
    
    recs.append(rec)

visualize_text(recs)
# The visualization is not available at github

True Label,Predicted Label (Prob),Target Label,Word Importance
1,1 (1.00),1,it ' s a charming and often affecting journey .
1,1 (0.96),1,the movie achieves as great an impact by keeping these thoughts hidden as . . . ( quills ) did by showing them .
0,0 (0.96),0,"this one is definitely one to skip , even for horror movie fanatics ."
0,0 (0.99),0,"in its best moments , resembles a bad high school production of grease , without benefit of song ."


## SmoothGradNLPInterpreter

In [8]:
recs = []
sg = it.SmoothGradNLPInterpreter(model, device='gpu:0')
for i, review in enumerate(reviews):
    sg_weights = sg.interpret(
        review,
        # tokenizer=tokenizer,
        text_to_input_fn=text_to_input,
        n_samples=50,
        noise_amount=0.01
    )
    pred_class = sg.predcited_label[0]
    pred_prob = sg.predcited_proba[0, pred_class]
    
    sg_weights = np.sum(np.abs(sg_weights), axis=-1)

    # subwords with [CLS] and [SEP]
    # encoded_inputs = tokenizer(review)
    # subwords = tokenizer.convert_ids_to_tokens(encoded_inputs['input_ids'])
    # subwords without special tokens.

    subwords = " ".join(tokenizer._tokenize(review)).split(' ')
    subword_importances = sg_weights[0]
    
    words, word_importances = aggregate_subwords_and_importances(subwords, subword_importances)
    word_importances = np.array(word_importances) / np.linalg.norm(
        word_importances)
    
    true_label = true_labels[i]
    interp_class = pred_class
    
    if interp_class == 0:
        word_importances = -word_importances
        
    rec = VisualizationTextRecord(
        words, 
        word_importances, 
        true_label,                   
        pred_class, 
        pred_prob,
        interp_class
    )
    
    recs.append(rec)

visualize_text(recs)
# The visualization is not available at github

100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 27.36it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 28.45it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 28.80it/s]


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [00:01<00:00, 26.01it/s]


True Label,Predicted Label (Prob),Target Label,Word Importance
1,1 (1.00),1,it ' s a charming and often affecting journey .
1,1 (0.96),1,the movie achieves as great an impact by keeping these thoughts hidden as . . . ( quills ) did by showing them .
0,0 (0.96),0,"this one is definitely one to skip , even for horror movie fanatics ."
0,0 (0.99),0,"in its best moments , resembles a bad high school production of grease , without benefit of song ."


## GradShapNLPInterpreter

In [4]:
# ig = it.GradShapNLPInterpreter(model, device='gpu:0')

# pred_labels, pred_probs, avg_gradients = ig.interpret(
#     preprocess_fn(data),
#     n_samples=10,
#     noise_amount=0.1,
#     return_pred=True)

# true_labels = [1, 1, 0, 0] * 5
# recs = []
# for i in range(avg_gradients.shape[0]):
#     subwords = " ".join(tokenizer._tokenize(data[i]['text'])).split(' ')
#     subword_importances = avg_gradients[i]
#     words, word_importances = aggregate_subwords_and_importances(subwords, subword_importances)
#     word_importances = np.array(word_importances) / np.linalg.norm(
#         word_importances)
    
#     pred_label = pred_labels[i]
#     pred_prob = pred_probs[i, pred_label]
#     true_label = true_labels[i]
#     interp_class = pred_label
    
#     if interp_class == 0:
#         word_importances = -word_importances
#     recs.append(
#         VisualizationTextRecord(words, word_importances, true_label,
#                                 pred_label, pred_prob, interp_class)
#     )

# visualize_text(recs)
# # The visualization is not available at github